# **Energy Benchmarking of Deep Learning Workflows**

### **1.) Introduction to Energy Benchmarking with CodeCarbon**

Researchers in the domains of Green AI or AI Efficiency have recenty started to include "Energy Fact Sheets" of the proposed models in their papers, similar to how the nutritional fact sheet on food items lists what is in the food you eat.

This makes it possible for others to compare and grasp what kind of resources the researchers had at their disposal (GPUs, TPUs, ...), how many parameters their models have (Amount and FLOPs), how much energy was needed for a single training iteration or a single inference call, and sometimes also how many re-training attempts they did in total during their development.

If direct hardware-based measuring of the energy usage is not possible or to much of an effort, software tools such as ***CodeCarbon*** are a good alternative, although generally less accurate. 

***CodeCarbon*** allows you to track the energy usage of your code with an intuitive *wrapping mechanism*. You can either use the explicit `EmissionsTracker` tracker object and wrap the code you want to benchmark with the `tracker.start()` and `tracker.stop()` functions. Or, if you already have your code bundled into individual functions, you can make use of the built-in function decorator `@track_emissions`.

Both versions do the same things: 
1. Keep track of the starting time when you call `tracker.start()` or the decorated function
2. Get an initial measurement of the energy usage at the start
3. Start a scheduler in the background that does a measurement every X seconds while your code runs
4. Wait until you either stop the tracking with `tracker.stop()` or the decorated function terminates
5. Do another meaurement at the end
6. Collect and aggregate the measurment data for you *(and store or return it to you depending on how you configured it)*


#### **The following two ways are examples of how you can use this in your code:**
##### **Version 1 - Tracker object**
```python
    from codecarbon import EmissionsTracker
    
    tracker = EmissionsTracker(
        #... configurations
    )

    tracker.start()
    try:
        # ... do something   
    finally:
        tracker.stop()
    
    results = tracker.final_emissions_data
```


##### **Version 2 - Function decorator**

```python
    from codecarbon import track_emissions

    @track_emissions(
        #... configurations
    )
    def func()
        # ... do something
```

To access the energy benchmark data, you can either look into the `.csv` file that is created by default. Every row corresponds to one call of `start()` to `stop()`. Alternatively, and more useful if you want to directly do something with the respective value, you can access the `final_emissions_data` attribute of the tracker object (This of course only works if you explicitly use the tracker object and not the function decorator).

### **2.) Energy Benchmarking of Model Training and Testing**

Understanding the energy usage of AI models is important for the reduction of the immense carbon footprint of modern AI and making AI development, in general, more sustainable. 

For this next part of the notebook, we will train deep learning models on the Oxford IIIT Pets dataset (see [here](https://pytorch.org/vision/stable/generated/torchvision.datasets.OxfordIIITPet.html) and [here](https://www.robots.ox.ac.uk/~vgg/data/pets/)). Your task is to train a deep learning model, achieve an accuracy of at least 80% and find ways to make the training phase of the model as energy efficient as possible.

We will start by implementing the two helper functions `train_model()` and `test_model()`. These should make it easier and faster for you to go over multiple iterations of your models and try different approaches. Once the basic functionality is implemented, you can extend them how ever you please.

#### **Getting Started**

1. Set up your virtual environment for python and run the import cell down below
2. Implement the leftout spaces of the train_model function
3. Implement a basic first model to test and familiarize yourself with how the two functions work (also helps to get a baseline for the accuracy and energy usage)
4. Extend your models and the two functions to increase the energy efficiency 
    - have a look at ***Things you can consider*** down below to get some ideas where to start.
    - have a look at the basic implementation of the `test_model()` function to get an understanding of what we expect from you in the `train_model()` function that you have to implement yourself.

**Things you can consider:**

- Run model on CPU vs. GPU (see [here](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html) and [here](https://pytorch.org/docs/stable/tensor_attributes.html#torch.device))
- Model architecture and size ([custom models](https://pytorch.org/tutorials/beginner/introyt/modelsyt_tutorial.html) vs. [built-in Pytorch models](https://pytorch.org/vision/stable/models.html))
- Pretrained model weights vs. starting from scratch
- Different optimizers (see [here](https://pytorch.org/docs/stable/optim.html))
- Learning rate and adaptation strategies (Warm-start, Decay, ...)
- Batch size
- Data augmentation (see [here](https://pytorch.org/vision/stable/transforms.html))
- Different loss functions
- Train/Test vs. Train/Validation/Test split of the data set. What could a additional validation set be useful for? (*Hint: Early Stopping*)

Due to the limited time you have, do not worry about implementing or trying out all of these examples. Start by discussing with your colleagues which of these examples help the model to be more energy efficient, and if so, how they achieve it.

Afterwards, pick and focus on the few that you think would help the most (or that you simply find the most interesting to implement). You are, of course, also allowed to come up with your own strategies.

In [1]:
# Dependencies
import os
import time
import copy

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torchvision.transforms import v2
from torch.utils.data import DataLoader, random_split

from codecarbon import EmissionsTracker

#### The `train_model()` function
This function should take in a PyTorch model, a string name for your model and a PyTorch DataLoader instance for the Oxford Pets dataset. In the end, it should take care of the complete model training process for you so that you can focus your energy on how and what to change in your models. (*Hint: as you might have seen in the **Things to consider** section, there are also a few things you can improve over basic the train_model() function to make the training phase a lot more efficient*)

In short, we want you to implement the standard deep learning training loop and afterwards expand it how ever you please:
- loop over the epochs and batches of input images and labels
- calculate the loss of your model predictions
- backpropagte the loss
- do the optimizer weight update step

Additionally, this is where you should utilize the ***CodeCarbon*** benchmarking library, to understand how much time and energy the training phase of your models need.


In [13]:
def train_model(model, model_name:str, train_loader:DataLoader, val_loader:DataLoader, num_epochs=20, threshold=0.9, lr=1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    loss_func = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Reduced learning rate when validation accuracy starts to plateau
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)

    tracker = EmissionsTracker(
        project_name="Model_Training",
        output_file="train_emissions.csv",
        measure_power_secs=20,
        log_level="critical",
        allow_multiple_runs=True
    )
    tracker.start()
    start_time = time.perf_counter()
    try:
        best_model_wts = copy.deepcopy(model.state_dict())
        best_acc = 0.0
        for epoch in range(num_epochs):
            running_loss = 0.0
            correct = total = 0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad() # Manually zero the accumulated gradients
                outputs = model(images)
                loss = loss_func(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
                
                images, labels = images.to("cpu"), labels.to("cpu")
            
            epoch_loss = running_loss / len(train_loader.dataset)
            epoch_acc = correct / total
            
            # Reduce learning rate when validation accuracy starts to plateau
            lr_scheduler.step()
            
            # Validation phase every 5th epoch
            if epoch % 5 == 4:
                model.eval()  # Set model to evaluation mode
                val_correct = 0
                val_total = 0
                
                with torch.no_grad():  # No gradients needed for validation
                    for images, labels in val_loader:
                        images, labels = images.to(device), labels.to(device)
                        
                        outputs = model(images)
                        _, predicted = torch.max(outputs, 1)
                        val_correct += (predicted == labels).sum().item()
                        val_total += labels.size(0)
                        images, labels = images.to("cpu"), labels.to("cpu")
                
                val_acc = val_correct / val_total
                print(f"  Epoch {epoch+1:3d}/{num_epochs} - Train Loss: {epoch_loss:.5f} | Train Accuracy: {100*epoch_acc:.1f}% | Validation Accuracy: {100*val_acc:.1f}%")
                
                # Save the best model based on validation accuracy
                if val_acc > best_acc:
                    best_acc = val_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                
                # If the validation accuracy reaches the threshold, stop training
                if val_acc >= threshold:
                    print(f"  Reached target validation accuracy of {threshold * 100}% at epoch {epoch+1}. Model training stopped.")
                    break

        print(f"  Trained for {epoch+1} epochs - Train Loss: {epoch_loss:.5f} | Train Accuracy: {100*epoch_acc:.1f}% | Validation Accuracy: {100*val_acc:.1f}%")

        training_time = time.perf_counter() - start_time
        
        tracker.stop()
        emissions_data = tracker.final_emissions_data

        print(f"[Training] Time: {training_time:.2f} seconds | CO2 Emissions: {emissions_data.emissions:.6f} kg | Total Energy Usage: {emissions_data.energy_consumed:.7f} kWh")
        
        # Load the best model weights
        model.load_state_dict(best_model_wts)

        # Store the model
        models_path = "./models/"
        os.makedirs(models_path, exist_ok=True)
        torch.save(model.state_dict(), models_path + model_name + f"_val-acc-{val_acc}")

        return model
    
    finally:
        tracker.stop()
        model.to("cpu")
        torch.cuda.empty_cache()

#### The `test_model()` function
This function again takes in a PyTorch model and a PyTorch DataLoader instance of the test split of the dataset, calculates the test accuracy and the inference latency and energy usage at test time of your models.

In [14]:
def test_model(model, test_dataloader):
    # Send model to gpu
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Set model to inference mode
    model.eval()

    tracker = EmissionsTracker(
        project_name="Model_Inference",
        output_file="inference_emissions.csv",
        measure_power_secs=20,
        log_level="error",
        allow_multiple_runs=True
    )
    tracker.start()

    start_time = time.perf_counter()
    try:
        correct = total = 0
        with torch.no_grad():
            for images, labels in test_dataloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                images, labels = images.to("cpu"), labels.to("cpu")
        
        total_inference_time = time.perf_counter() - start_time

        avg_inference_latency = total_inference_time / len(test_dataloader.dataset)
        test_acc = 100 * correct / total
        
        tracker.stop()
        emissions_data = tracker.final_emissions_data

        print(f"[Inference] #Test Samples: {len(test_dataloader.dataset)} | Average Inference Latency: {avg_inference_latency:.4f} seconds | Test Accuracy: {test_acc:.1f}%") 
        print(f"  - CO2 Emissions: {emissions_data.emissions:.6f} kg | Total Energy Usage: {emissions_data.energy_consumed:.7f} kWh")
        return {
            "inference_latency": avg_inference_latency,
            "test_accuracy": test_acc,
            "emissions_data": emissions_data
        }

    finally:
        tracker.stop()
        model.to("cpu")
        torch.cuda.empty_cache()

#### Finally create, train and test your models here

In [15]:
batch_size = 64
output_size = 37 # Oxford Pets 37 categories

# Data set transforms for the Oxford Pets Dataset
transforms = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.Resize((224,224)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
os.makedirs("./data", exist_ok=True)
train_dataset_complete = datasets.OxfordIIITPet(root="./data", split="trainval", transform=transforms, download=True)
test_dataset = datasets.OxfordIIITPet(root="./data", split="test", transform=transforms, download=True)

# Split train dataset into train and validation
train_dataset, validation_dataset = random_split(train_dataset_complete, [0.9, 0.1])

# Setup data loaders to batch image and labels
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
### Create your model
    # You can either build one from scratch using a custom layer setup, or try builtin model architectures from PyTorch (ResNet, MobileNet, ...)
    # Best start with a simple model to test the training and testing workflow and then work up to more complex models
print("### Training ResNet34 with pretrained weights")
resnet34_pre = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
resnet34_pre.fc = nn.Linear(resnet34_pre.fc.in_features, output_size)

resnet34_pre = train_model(resnet34_pre, "resnet_34_pre", train_dataloader, validation_dataloader, lr=1e-4, num_epochs=20, threshold=0.99)


### Training ResNet34 with pretrained weights
  Epoch   5/20 - Train Loss: 0.05675 | Train Accuracy: 99.6% | Validation Accuracy: 95.7%
  Epoch  10/20 - Train Loss: 0.00059 | Train Accuracy: 100.0% | Validation Accuracy: 94.6%
  Epoch  15/20 - Train Loss: 0.00034 | Train Accuracy: 100.0% | Validation Accuracy: 95.1%
  Epoch  20/20 - Train Loss: 0.00033 | Train Accuracy: 100.0% | Validation Accuracy: 95.4%
  Trained for 20 epochs - Train Loss: 0.00033 | Train Accuracy: 100.0% | Validation Accuracy: 95.4%
[Training] Time: 418.10 seconds | CO2 Emissions: 0.007911 kg | Total Energy Usage: 0.0207654 kWh


In [ ]:
results = test_model(resnet34_pre, test_dataloader)

resnet34_pre.to("cpu")
del resnet34_pre
torch.cuda.empty_cache()

[Inference] #Test Samples: 3669 | Average Inference Latency: 0.0054 seconds | Test Accuracy: 89.2%
  - CO2 Emissions: 0.000259 kg | Total Energy Usage: 0.0006798 kWh
